# Overdue HW 4 (oops :[ )


## Exercises

You may work in groups for these exercises. They are due at the beginning of next class. You can submit them as a link to a Colab notebook or GitHub CodeSpace.

0. Use the corpora that you assembled last week (Pausanias++):
1. Using programming techniques from the course so far, find other potential collocates for a word of your choice.
2. Calculate the μ and Mutual Information scores for at least 5 of these collocate pairs. How do your results change depending on your definition of a collocation? What might these changes mean? (Write your answers to these questions down.)
3. Calculate the Delta P for these same five pairs. Do any results stand out? Why? What might they tell us about your corpus.


## 0

In [3]:
%pip install -r requirements.txt

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
Using cached certifi-2024.8.30-py3-none-any.whl (167 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 562.4/562.4 kB 19.2 MB/s eta 0:00:00
  Created wheel for MyCapytain: filename=MyCapytain-3.0.2-py3-none-any.whl size=79676 sha256=c330d8c63a0779b2e40e70622511f19d985d37fc391a87e81eb96d2bc3028837
  Stored in directory: /home/codespace/.cache/pip/wheels/fa/bd/e8/b00dcc7f35402008e260036a3889b274b460ed8a954084ae17
  Created wheel for LinkHeader: filename=LinkHeader-0.4.3-py3-none-any.whl size=4787 sha256=cbd51dbd6c076b2ca2b3d7b387c3c984c6572ea712ad4b323ca00697c44782f8
  Stored in directory: /home/codespace/.cache/pip/wheels/cd/58/bd/50fb033e129108ae37dd8891dc93f80de60a61866c6580bbf6
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26303 sha256=eaba4c5e68c00581c1983ef241c1d29d73e

In [5]:
from lxml import etree
from MyCapytain.common.constants import Mimetypes
from MyCapytain.resources.texts.local.capitains.cts import CapitainsCtsText
import pandas as pd

with open("/tei/tlg0525.tlg001.perseus-grc2.xml") as f:
    text = CapitainsCtsText(urn="urn:cts:greekLit:tlg0525.tlg001.perseus-grc2", resource=f)

urns = []
raw_xmls = []
unannotated_strings = []

for ref in text.getReffs(level=len(text.citation)):
    urn = f"{text.urn}:{ref}"
    node = text.getTextualNode(ref)
    raw_xml = node.export(Mimetypes.XML.TEI)
    tree = node.export(Mimetypes.PYTHON.ETREE)
    s = etree.tostring(tree, encoding="unicode", method="text")

    urns.append(urn)
    raw_xmls.append(raw_xml)
    unannotated_strings.append(s)

d = {
    "urn": pd.Series(urns, dtype="string"),
    "raw_xml": raw_xmls,
    "unannotated_strings": pd.Series(unannotated_strings, dtype="string")
}
pausanias_df = pd.DataFrame(d)
pausanias_df['whitespaced_tokens'] = pausanias_df['unannotated_strings'].str.split()
pausanias_df

FileNotFoundError: [Errno 2] No such file or directory: '/tei/tlg0525.tlg001.perseus-grc2.xml'

## 1

## 2

## 3